In [4]:
pip check

    pandas (>=0.25.*)
            ~~~~~~~^
conda-verify 3.4.2 is not supported on this platform
grpcio 1.51.3 is not supported on this platform
tensorflow-macos 2.11.0 is not supported on this platform
xgboost 1.7.4 is not supported on this platform
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import scipy
import numba
import matplotlib
import seaborn

print("NumPy version:", np.__version__)
print("SciPy version:", scipy.__version__)
print("Numba version:", numba.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("Seaborn version:", seaborn.__version__)


NumPy version: 1.24.4
SciPy version: 1.13.1
Numba version: 0.60.0
Matplotlib version: 3.9.2
Seaborn version: 0.13.2


In [2]:
import pandas as pd
import numpy as np
import wrangle as w

import seaborn as sns
import matplotlib.pyplot as plt
import time

In [3]:
df = w.clean_data('nfl_data.csv')
df

,season,team,week,day,date,result,ot,host,opp,pts,...,fga_opp,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,0,3,3,6,155,7,14,0,0,31:30
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,1,4,4,2,23,5,12,3,4,28:40
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,5,2,2,2,164,5,12,0,0,36:28
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,...,2,3,3,6,137,7,16,0,0,32:10
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,...,1,3,3,7,206,5,14,0,1,28:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12345,2023,WAS,13,Sun,December 3,L,no,home,Miami Dolphins,15,...,1,6,6,3,127,7,13,2,2,32:16
12346,2023,WAS,15,Sun,December 17,L,no,away,Los Angeles Rams,20,...,3,2,2,3,134,9,16,0,0,35:53
12347,2023,WAS,16,Sun,December 24,L,no,away,New York Jets,28,...,3,3,3,6,292,4,16,1,2,36:16
12348,2023,WAS,17,Sun,December 31,L,no,home,San Francisco 49ers,10,...,2,3,3,2,76,4,9,0,1,38:13


In [5]:
df.shape

(12350, 63)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12350 entries, 0 to 12349
Data columns (total 63 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                12350 non-null  int64  
 1   team                  12350 non-null  object 
 2   week                  12350 non-null  int64  
 3   day                   12350 non-null  object 
 4   date                  12350 non-null  object 
 5   result                12350 non-null  object 
 6   ot                    12350 non-null  object 
 7   host                  12350 non-null  object 
 8   opp                   12350 non-null  object 
 9   pts                   12350 non-null  int64  
 10  opp_pts               12350 non-null  int64  
 11  pass_cmp              12350 non-null  int64  
 12  pass_att              12350 non-null  int64  
 13  pass_yds              12350 non-null  int64  
 14  pass_td               12350 non-null  int64  
 15  int                

In [9]:
df.iloc[:16]

,season,team,week,day,date,result,ot,host,opp,pts,...,fga_opp,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp
0,2000,CRD,1,Sun,September 3,L,no,away,New York Giants,16,...,0,3,3,6,155,7,14,0,0,31:30
1,2000,CRD,2,Sun,September 10,W,no,home,Dallas Cowboys,32,...,1,4,4,2,23,5,12,3,4,28:40
2,2000,CRD,4,Sun,September 24,L,no,home,Green Bay Packers,3,...,5,2,2,2,164,5,12,0,0,36:28
3,2000,CRD,5,Sun,October 1,L,no,away,San Francisco 49ers,20,...,2,3,3,6,137,7,16,0,0,32:10
4,2000,CRD,6,Sun,October 8,W,no,home,Cleveland Browns,29,...,1,3,3,7,206,5,14,0,1,28:57
5,2000,CRD,7,Sun,October 15,L,no,home,Philadelphia Eagles,14,...,2,3,4,3,42,10,18,2,2,39:34
6,2000,CRD,8,Sun,October 22,L,no,away,Dallas Cowboys,7,...,2,6,6,3,38,3,7,0,0,30:45
7,2000,CRD,9,Sun,October 29,L,no,home,New Orleans Saints,10,...,1,3,3,5,118,6,14,0,0,26:41
8,2000,CRD,10,Sun,November 5,W,no,home,Washington Redskins,16,...,5,0,0,0,0,8,15,0,1,37:26
9,2000,CRD,11,Sun,November 12,L,no,away,Minnesota Vikings,14,...,1,4,4,1,39,6,9,0,1,34:08


In [21]:
df[(df['season'] == 2000) & (df['team'].str.contains('nyg', case=False))]

,season,team,week,day,date,result,ot,host,opp,pts,...,fga_opp,xpm_opp,xpa_opp,pnt_opp,punt_yds_opp,3dconv_opp,3datt_opp,4dconv_opp,4datt_opp,top_opp
352,2000,NYG,1,Sun,September 3,W,no,home,Arizona Cardinals,21,...,1,1,1,5,105,6,15,0,2,28:30
353,2000,NYG,2,Sun,September 10,W,no,away,Philadelphia Eagles,33,...,2,0,1,4,82,3,10,0,1,21:16
354,2000,NYG,3,Sun,September 17,W,no,away,Chicago Bears,14,...,1,1,1,7,211,3,12,0,1,21:49
355,2000,NYG,4,Sun,September 24,L,no,home,Washington Redskins,6,...,2,1,2,5,88,5,14,0,0,31:22
356,2000,NYG,5,Sun,October 1,L,no,away,Tennessee Titans,14,...,1,4,4,4,66,14,20,0,1,42:46
357,2000,NYG,6,Sun,October 8,W,no,away,Atlanta Falcons,13,...,3,0,0,5,92,5,14,0,1,30:15
358,2000,NYG,7,Sun,October 15,W,no,home,Dallas Cowboys,19,...,0,2,2,3,39,5,13,0,1,30:42
359,2000,NYG,9,Sun,October 29,W,no,home,Philadelphia Eagles,24,...,1,1,1,9,300,1,11,0,1,16:19
360,2000,NYG,10,Sun,November 5,W,no,away,Cleveland Browns,24,...,1,0,0,6,176,5,13,0,1,23:20
361,2000,NYG,11,Sun,November 12,L,no,home,St. Louis Rams,24,...,2,5,5,6,144,3,13,1,1,36:40
